In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
import time

# generate url for product
def get_url(product):
    product = product.replace(' ','%20') #open shopee and search an item and can see that wide space replace by %20
    template = 'https://shopee.com.my/search?keyword={}'
    url = template.format(product)
    return url

def get_all_products(card):
    pImg = card.find('img')
    product_image = pImg['src']
    product_name = card.find('div','_10Wbs- _5SSWfi UjjMrh').text.strip()
    product_name = product_name.encode('ascii','ignore')  #remove emoji from line 19 to line 20
    product_name = str(product_name,'utf-8')
    product_price = card.find('div','zp9xm9 xSxKlK _1heB4J').text.strip()
    anchor_tag = card.a.get('href')
    product_buy_link = 'https://shopee.com.my'+anchor_tag
    
    product_info = (product_image,product_name,product_price,product_buy_link)
    
    return product_info

def main(product):
    records = []
    url = get_url(product)
    
    driver = webdriver.Chrome(executable_path='chromedriver.exe')
    driver.get(url)
    driver.maximize_window()
    # wait for 5 seconds
    time.sleep(5)
    btn = driver.find_element_by_xpath('//*[@id="modal"]/div[1]/div[1]/div/div[3]/div[1]/button')
    btn.click()


    #Define an initial value   #link: https://wwwhttps://www.programmersought.com/article/10507414140/.programmersought.com/article/10507414140/
    temp_height=0
    
    while True:
        #Looping down the scroll bar
        driver.execute_script("window.scrollBy(0,1000)")
        #sleep and let the scroll bar react
        time.sleep(5)
        #Get the distance of the current scroll bar from the top
        check_height = driver.execute_script("return document.documentElement.scrollTop || window.pageYOffset || document.body.scrollTop;")
        #If the two are equal to the end
        if check_height==temp_height:
            break
        temp_height=check_height
    
    time.sleep(10)      #wait for awhile as the page not fully loaded

    # create a BeautifulSoup object
    soup = BeautifulSoup(driver.page_source , 'html.parser')
    product_cards = soup.find_all('div','col-xs-2-4')
    
    for everyCard in product_cards:
        productDetails = get_all_products(everyCard)
        records.append(productDetails)

    # Here We Are Using Pandas Data Frame To Save All The Product Information In A CSV File
    
    col = ['Product_Image','Product_Name','Product_Price','Product_Buy_Link']
    
    shopee_data = pd.DataFrame(records,columns=col)
    
    shopee_data.to_csv('D:\\ShopeeData.csv',encoding='utf-8')
        

In [2]:
product = input('Enter Product Looking For : ')
main(product)

Enter Product Looking For : ram card
